# SparseEdges


## A bio-inspired sparse representation of edges in natural images


Table of content

* [What is the SparseEdges package?](#What-is-the-SparseEdges-package?) 
* [Installing](#Installing) 
* [testing one step](#testing-one-step)


What is the SparseEdges package?
================================

Our goal here is to build practical algorithms of sparse coding for computer vision.

The code is available @ https://github.com/bicv/SparseEdges

This class exploits the LogGabor package to provide with a sparse representation of edges in images.

This algorithm was presented in the following paper:

```bibtex
@inbook{Perrinet15bicv,
    title = {Sparse models},
    author = {Perrinet, Laurent U.},
    booktitle = {Biologically-inspired Computer Vision},
    chapter = {13},
    editor = {Keil, Matthias and Crist\'{o}bal, Gabriel and Perrinet, Laurent U.},
    publisher = {Wiley, New-York},
    year = {2015}
}
```



This package gives a python implementation.

Moreover, it gives additional tools to compute useful stistics in images; first- and second order statistics of co-occurences in images.
More information is available @ http://nbviewer.ipython.org/github/bicv/SparseEdges/blob/master/SparseEdges.ipynb
Tests for the packages are available @ http://nbviewer.ipython.org/github/bicv/SparseEdges/blob/master/test-SparseEdges.ipynb.

## Installing

To install the ``SparseEdges`` class, you would need the ``LogGabor`` class (multiscale filters) which itself depends on the ``SLIP`` class (the image processing tools).

    pip install git+https://github.com/bicv/SLIP.git
    pip install git+https://github.com/bicv/LogGabor.git 
    pip install git+https://github.com/bicv/SparseEdges.git
    

But before, you need th usual depndencies, such as numpy, matplotlib, pyprind and imageio:

    pip install -U numpy matplotlib pyprind imageio


[Back to top](#SparseEdges)

In [ ]:
%cd -q test
from __future__ import division, print_function
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=4)#, suppress=True)

### Extracting edges on a sample image

We will show here how we can simply reconstruct an example image with the list of extracted edges overlaid.

First we define our object by loading default parameters from internet:

In [ ]:
from SparseEdges import SparseEdges
mp = SparseEdges('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
print(mp.pe)

At this point, we can change these parameters, by instance by using ``2048`` edges and a different value for the $\alpha$ value in matching pursuit:

In [ ]:
mp.pe.N = 16
mp.pe.MP_alpha = .9

We can now load an image and make sure to set the framework to the appropriate size:

In [ ]:
# defining input image
name = 'example'
image = mp.imread('https://raw.githubusercontent.com/bicv/SparseEdges/master/database/lena256.png')
mp.set_size(image)

... then, we can initialize the algorithm and normalize the image:

In [ ]:
mp.init()
image = mp.normalize(image, center=True)
print(image.mean(), image.std())

Then, it is easy to run matching pursuit on that image (or load a cached file with the results):

In [ ]:
print(os.path.join(mp.pe.matpath, name + '.npy'))

In [ ]:
try:
    edges = np.load(os.path.join(mp.pe.matpath, name + '.npy'))
except Exception:
    edges, C_res = mp.run_mp(image, verbose=True)
    np.save(mp.pe.matpath, name + '.npy', edges)    

Let's summarize that in one script:

In [ ]:
%%writefile experiment_example.py
#! /usr/bin/env python
# -*- coding: utf8 -*-
from __future__ import division, print_function
"""

An example MP run.

To run:
$ python experiment_example.py 

To remove cache:
$ rm -fr **/example*

"""
__author__ = "(c) Laurent Perrinet INT - CNRS"


import numpy as np
from SparseEdges import SparseEdges
mp = SparseEdges('https://raw.githubusercontent.com/bicv/SparseEdges/master/default_param.py')
mp.N = 128

image = mp.imread('https://raw.githubusercontent.com/bicv/SparseEdges/master/database/lena256.png')

name = 'example'
image = mp.normalize(image, center=True)
#print image.mean(), image.std()
white = mp.whitening(image)
white *= mp.mask


import os
matname = os.path.join(mp.pe.matpath, name + '.npy')
try:
    edges = np.load(matname)
except Exception:
    edges, C_res = mp.run_mp(white, verbose=True)
    np.save(matname, edges)    


In [ ]:
%run experiment_example.py

Let's show the results of the sparse edge extraction with the edges overlaid on the original image:

In [ ]:
mp.pe.figsize_edges = 12
mp.pe.line_width = 3.
mp.pe.scale = .5

fig, a = mp.show_edges(edges, image=mp.dewhitening(mp.whitening(image)), show_phase=False, mask=True)

Note the dashed circle which (as in Geisler, 2001) shows the limit after which we discard edges. Indeed, when computing statistics (our main goal) we wish to be not perturbed by the fact that images are rectangular.

Let's show the results of the sparse edge extraction with the edges overlaid on the image reconstructed from the edges:

In [ ]:
image_rec = mp.reconstruct(edges, mask=True)        
fig, a = mp.show_edges(edges, image=mp.dewhitening(image_rec), show_phase=False, mask=True)


Check out [this blog post](http://blog.invibe.net/posts/2015-05-22-a-hitchhiker-guide-to-matching-pursuit.html) to learn more about Matching Pursuit.

## more examples
First, some tests are available as separate notebooks:

In [ ]:
from IPython.display import FileLink, FileLinks, Image
FileLinks('../notebooks', recursive=False)

[Back to top](#SparseEdges)

### Effect of parameters on edge extraction: image size


In [ ]:
# TODO include figure

### Effect of parameters on edge extraction: filter parameters



In [ ]:
# TODO include figure

As we test different parameters for the filters, we measured the gain in efficiency for the algorithm as the ratio of the code length to achieve $85\%$ of energy extraction relative to that for the default parameters (white bar). The average is computed on the same database of natural images and error bars denote the standard deviation of gain over the database. First, we studied the effect of the bandwidth of filters respectively in the $\textsf{(A)}$ spatial frequency and $\textsf{(B)}$ orientation spaces. The minimum is reached for the default parameters: this shows that default parameters provide an optimal compromise between the precision of filters in the frequency and position domains for this database. We may also compare pyramids with different number of filters.  Indeed, efficiency (in bits) is equal to the number of selected filters times the coding cost for the address of each edge in the pyramid.
We plot here the average gain in efficiency which shows an optimal compromise respectively for respectively $\textsf{(C)}$ the number of orientations and $\textsf{(D)}$ the number of spatial frequencies (scales). Note first that with more than 12 directions, the gain remains stable. Note also that a dyadic scale ratio (that is of 2) is efficient but that other solutions ---such as using the golden section $\phi$--- prove to be significantly more efficient, though the average gain is relatively small (inferior to $5\%$).

## some book keeping for the notebook

In [ ]:
%load_ext watermark
%watermark

In [ ]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, sympy

In [ ]:
%cd -q ..